In [108]:
# input
n = 4
c = (2,2,3,2)       # the number of kWh of electricity you consumed this day
p = (2,5,8,10)      # the variable price per kWh, what you would pay this day if you have no fixed plan.
q = (4,3,4,4)       # the price per kWh for a 1-year plan started this day.
r = (10,9,8,7)      # the price per kWh for a 2-year plan started this day. 
s = (10,10,10,10)   # the price per kWh for a 5-year plan started this day.

In [109]:
def P(x):
    global n, memo
    n = x
    memo = {}
    return p1(0)

def p1(d):
    if d > n:
        return 10**100
    elif d == n:
        return 0

    if d in memo:
        return memo[d]

    q1 = 365 if n-d > 365 else n-d
    r1 = 730 if n-d > 730 else n-d
    s1 = 1825 if n-d > 1825 else n-d

    result = min(c[d]*p[d] + p1(d+1), sum(c[d:d+q1])*q[d] + p1(d+q1), sum(c[d:d+r1])*r[d] + p1(d+r1), sum(c[d:d+s1])*s[d] + p1(d+s1))

    memo[d] = result
    return result


In [110]:
P(4)

25

In [139]:
def smallest_total_cost():
    var_cost = [0] * n
    one_year_cost = [0] * n
    two_year_cost = [0] * n
    five_year_cost = [0] * n
    dp = [[0] * 4 for _ in range(n)]

    var_cost[0] = p[0]
    one_year_cost[0] = q[0]
    two_year_cost[0] = r[0]
    five_year_cost[0] = s[0]

    for i in range(1, n):
        var_cost[i] = min(p[i], var_cost[i-1] + c[i] * p[i])
        one_year_cost[i] = min(q[i] + dp[i-1][0], one_year_cost[max(0, i-365)] + q[max(0, i-365)] + sum(c[max(0, i-364):i+1]) * q[max(0, i-365)])
        two_year_cost[i] = min(r[i] + dp[i-1][1], two_year_cost[max(0, i-730)] + r[max(0, i-730)] + sum(c[max(0, i-729):i+1]) * r[max(0, i-730)])
        five_year_cost[i] = min(s[i] + dp[i-1][2], five_year_cost[max(0, i-1825)] + s[max(0, i-1825)] + sum(c[max(0, i-1824):i+1]) * s[max(0, i-1825)])
        dp[i][0] = min(var_cost[i], one_year_cost[i], two_year_cost[i], five_year_cost[i])
        dp[i][1] = min(one_year_cost[max(0, i-365)], two_year_cost[max(0, i-730)], five_year_cost[max(0, i-1825)])
        dp[i][2] = min(two_year_cost[max(0, i-730)], five_year_cost[max(0, i-1825)])
        dp[i][3] = five_year_cost[max(0, i-1825)]

    return min(dp[n-1][0], dp[n-1][1], dp[n-1][2], dp[n-1][3])


In [140]:
smallest_total_cost()

4

In [142]:
def optimal_plan(n, c, p, q, r, s):
    dp = [[float('inf'), float('inf'), float('inf')] for _ in range(n)]
    dp[0][0] = c[0] * p[0]
    one_year_cost = [q[i] + sum(c[:i+1]) * q[i] for i in range(n)]
    two_year_cost = [r[i] + sum(c[:i+1]) * r[i] for i in range(n)]
    five_year_cost = [s[i] + sum(c[:i+1]) * s[i] for i in range(n)]
    for i in range(1, n):
        dp[i][0] = min(dp[i-1][0] + c[i] * p[i], one_year_cost[i], two_year_cost[i], five_year_cost[i])
        if i >= 365:
            one_year_cost[i] = min(q[i] + dp[i-365][0], one_year_cost[i-365] + q[i] + sum(c[i-364:i+1]) * q[i])
        if i >= 730:
            two_year_cost[i] = min(r[i] + dp[i-730][1], two_year_cost[i-730] + r[i] + sum(c[i-729:i+1]) * r[i])
        if i >= 1825:
            five_year_cost[i] = min(s[i] + dp[i-1825][2], five_year_cost[i-1825] + s[i] + sum(c[i-1824:i+1]) * s[i])

    return dp[-1][0]

# Example usage:
n = 4
c = [2, 2, 3, 2]
p = [2, 5, 8, 10]
q = [4, 3, 4, 4]
r = [10, 9, 8, 7]
s = [10, 10, 10, 10]
print(optimal_plan(n, c, p, q, r, s)) # Output: 25


[[4, inf, inf], [14, inf, inf], [32, inf, inf], [40, inf, inf]]
40
